<a href="https://colab.research.google.com/github/blanco-herrero/Interviews/blob/main/TOPIC_MODELLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing modules
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim import corpora
from gensim import models
from gensim import matutils
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
os.chdir('..')
from glob import glob
from string import punctuation
import nltk
from nltk.sentiment import vader
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Upload the file
from google.colab import files
files.upload()

In [3]:
# Read data into an object
entrevistas_df = pd.read_csv("Entrevistas.txt", header= None, sep='\t')
entrevistas = entrevistas_df.T.squeeze()

In [4]:
# Print out the first rows of the file
entrevistas

0      Sí, claro. Yo creo que están en el día a día, ...
1      Bueno, tendría que hacer memoria. La verdad es...
2      Bueno, la verdad es que cuando dedicas espacio...
3      La verdad es que hay, esto tiene que ver con l...
4      Bueno, los que buscan enemigos, los que simpli...
                             ...                        
519    Honestamente, es un artículo que espero que le...
520    Con las estadísticas, por ejemplo, me he hecho...
521    Sí, escuchar, escuchar, escuchar. Como decían ...
522    La movida gore, que yo no la meto normalmente ...
523                                                   21
Name: 0, Length: 524, dtype: object

In [81]:
# Remove the columns (if the case)
# trans = trans.drop(columns=['id', 'location', 'language'], axis=1).sample(100)

In [6]:
import re
(lambda x: re.sub('[-–_–,:/.¡!¿?...]','', x))

<function __main__.<lambda>(x)>

In [7]:
# Remove punctuation
entrevistas = entrevistas.map(lambda x: re.sub('[-–_–,:/.¡!¿?...]','', x))
entrevistas

0      Sí claro Yo creo que están en el día a día evi...
1      Bueno tendría que hacer memoria La verdad es q...
2      Bueno la verdad es que cuando dedicas espacio ...
3      La verdad es que hay esto tiene que ver con la...
4      Bueno los que buscan enemigos los que simplifi...
                             ...                        
519    Honestamente es un artículo que espero que lea...
520    Con las estadísticas por ejemplo me he hecho c...
521    Sí escuchar escuchar escuchar Como decían Faem...
522    La movida gore que yo no la meto normalmente e...
523                                                   21
Name: 0, Length: 524, dtype: object

In [8]:
# Convert uppercase to lowercase
entrevistas.map(lambda x: x.lower())

0      sí claro yo creo que están en el día a día evi...
1      bueno tendría que hacer memoria la verdad es q...
2      bueno la verdad es que cuando dedicas espacio ...
3      la verdad es que hay esto tiene que ver con la...
4      bueno los que buscan enemigos los que simplifi...
                             ...                        
519    honestamente es un artículo que espero que lea...
520    con las estadísticas por ejemplo me he hecho c...
521    sí escuchar escuchar escuchar como decían faem...
522    la movida gore que yo no la meto normalmente e...
523                                                   21
Name: 0, Length: 524, dtype: object

In [72]:
# PREPARE DATA FOR LDA ANALYSIS

# Generate the stop words
stop_words = stopwords.words('spanish')

stop_words.extend(['http', 'https', 'tco', 'httpstco', 'co', 'pa', 'si', 
                   'rt', 'etc', 'ect', 'tco', 'xa0', 't', 'si', 'q', 'd', 'lo', 'ell', 'call', 
                   'pkly', 'onde', 'pese', 'tb', '000', 'dos', 'tres', 'cia', 'vez', 'ves', 
                   'mira', 'tan', 'tal', 'dar', 'da', 'das', 'dan', 'uma', 'va', 'van', 'ser',
                   'ahí', 'ahi', 'tras', 'detrás', 'detras', 'creo', 'hecho', 'ejemplo', 'demas', 
                   'demás', 'ademas', 'además', 'resto', 'pensar', 'sino', 'decir', 'lado', 
                   'parece', 'piensa', 'pase', 'pesar', 'entender', 'alguien', 'dicho', 'supuesto', 
                   'alli', 'allí', 'aun', 'cualquier', 'cuestión', 'contenido', 'contenidos', 
                   'simplemente', 'símplemente', 'habia', 'parte', 'acaso', 'biden', 'toda', 'todas',
                   'gracias', 'despues', 'después', 'of', 'and', 'his', 'the', 'cada', 'to', 
                   'in', 'ello', 'quiere', 'buenas', 'ningún', 'día', 'pocos', 'cómo', 'como', 'puede', 
                   'pone', 'mientras', 'garcimoreno', 'cabrita', 'veintimillapier', 'qls', 'colocolo', 
                   'gyzytqqtbm', 'sólo', 'solo', 'ahora', 'frente', 'hacen', 'hace', 'bien', 'años', 
                   'nueva', 'luego', 'así', 'asi','claro', 'ver', 'debería', 'video', 'estan', 'mas', 
                   'menos', 'hacer', 'solo', 'pues', 'incluso', 'meses', 'vale', 'dia', 'evidentemente', 
                   'tambien', 'entonces','bueno', 'gente', 'discurso', 'aqui', 'veces', 'digo', 'vamos', 
                   'momento', 'forma', 'cosas', 'tipo', 'siempre', 'caso', 'persona', 'personas', 'puedes', 
                   'quiza', 'pasa', 'tema', 'alguna', 'algunas', 'manera', 'veo', 'final', 'nunca', 'muchas', 
                   'veces', 'igual', 'quizas', 'dice', 'tener', 'hacia', 'digamos', 'bastante', 'mucha', 
                   'tampoco', 'tambien', 'mismo', 'decia', 'voy', 'cosa', 'ambito', 'aunque', 'algun',
                   'puedo', 'haber', 'quiero', 'pueden', 'mejor', 'lleva', 'dicen', 'depende', 'sido',
                   'general'])

len(stop_words)

503

In [ ]:
stop_words

In [44]:
#spanish_stopwords.append('')
#spanish_stopwords.remove('')

In [73]:
def sent_to_words(sentences):
    for sentence in sentences:
        
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]  

In [74]:
data = entrevistas.values.tolist()
data_words = list(sent_to_words(data))

In [75]:
# remove stop words
data_words = remove_stopwords(data_words)

In [48]:
print(data_words[:1][0][:30])

['verdad', 'buscan', 'chivos', 'expiatorios', 'exacerbada', 'espana', 'pais', 'curioso', 'racismo', 'espana', 'disimulamos', 'disfrazamos', 'rascas', 'aparece', 'hablas', 'lenguaje', 'fundamental', 'traduce', 'precisamente', 'especie', 'pensamiento', 'subterraneo', 'terminologia', 'respecto', 'habla', 'invasion', 'habla', 'avalancha', 'terminologia', 'completamente']


In [76]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
# LDA model training

from pprint import pprint
# Number of topics
num_topics = 4

# Build LDA model
lda_model = models.ldamodel.LdaModel(num_topics=num_topics, corpus=corpus, id2word=id2word, random_state=None, 
                                     per_word_topics=True, alpha='auto', passes=2)
lda_model.print_topics(num_words=10)

In [ ]:
# Print the keywords of the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
lda_model.print_topics(num_words=40)


In [ ]:
# Analyze LDA model results
!pip install pyLDAvis
!python -m pip install -U pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 

In [79]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

In [80]:
# This is a bit time consuming - make the if statement True
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [81]:
# Load the prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.015262 -0.012693       1        1  34.923688
1      0.035498  0.000735       2        1  24.031459
3     -0.032049 -0.029344       3        1  22.469806
0     -0.018711  0.041302       4        1  18.575047, topic_info=             Term        Freq       Total Category  logprob  loglift
858   informacion  259.000000  259.000000  Default  30.0000  30.0000
480          odio  369.000000  369.000000  Default  29.0000  29.0000
1952      twitter   95.000000   95.000000  Default  28.0000  28.0000
2265      conozco   34.000000   34.000000  Default  27.0000  27.0000
1533     escribir   36.000000   36.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
281        medios   18.835444  160.876579   Topic4  -6.2041  -0.4615
381         temas   17.331268  123.081834   Topic4  -6.2874  -0.2770
121          anos   17.052772  120.744977   Topic4  -6.3036  -0.2740
168        contar   16.087384  109.577651   Topic4  -6.3618  -0.2352
293         mundo   15.150497  113.715688   Topic4  -6.4218  -0.3323

[371 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
6022      1  0.200040           abril
6022      3  0.600121           abril
7075      2  0.807470  acompanamiento
7498      1  0.135106            acto
7498      2  0.675531            acto
...     ...       ...             ...
1286      4  0.078581             voz
2705      1  0.160551          vuelta
2705      2  0.160551          vuelta
2705      3  0.642204          vuelta
2705      4  0.080276          vuelta

[761 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1])

In [ ]:
# Take a look to the texts and their topics
topics = pd.DataFrame([dict(lda_model.get_document_topics(doc, minimum_probability=0.1))
                      for doc in corpus])
meta = entrevistas.iloc[entrevistas.index].drop(columns=[0]).reset_index(drop=True)
tpd = pd.concat([meta, topics], axis=1)
tpd [:50]